In [11]:
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

(0, array([ 0.44723853], dtype=float32), array([ 0.14281181], dtype=float32))
(20, array([ 0.17864653], dtype=float32), array([ 0.25558233], dtype=float32))
(40, array([ 0.11871085], dtype=float32), array([ 0.28943259], dtype=float32))
(60, array([ 0.10445153], dtype=float32), array([ 0.29748589], dtype=float32))
(80, array([ 0.10105906], dtype=float32), array([ 0.29940188], dtype=float32))
(100, array([ 0.10025196], dtype=float32), array([ 0.29985771], dtype=float32))
(120, array([ 0.10005995], dtype=float32), array([ 0.29996616], dtype=float32))
(140, array([ 0.10001425], dtype=float32), array([ 0.29999197], dtype=float32))
(160, array([ 0.1000034], dtype=float32), array([ 0.2999981], dtype=float32))
(180, array([ 0.10000079], dtype=float32), array([ 0.29999956], dtype=float32))
(200, array([ 0.1000002], dtype=float32), array([ 0.29999989], dtype=float32))


In [29]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 156s - loss: 0.3733 - acc: 0.8857 - val_loss: 0.0883 - val_acc: 0.9717
Epoch 2/12
60000/60000 [==============================] - 167s - loss: 0.1348 - acc: 0.9597 - val_loss: 0.0632 - val_acc: 0.9797
Epoch 3/12
60000/60000 [==============================] - 169s - loss: 0.1025 - acc: 0.9695 - val_loss: 0.0557 - val_acc: 0.9824
Epoch 4/12
60000/60000 [==============================] - 169s - loss: 0.0884 - acc: 0.9742 - val_loss: 0.0439 - val_acc: 0.9857
Epoch 5/12
60000/60000 [==============================] - 169s - loss: 0.0780 - acc: 0.9776 - val_loss: 0.0414 - val_acc: 0.9867
Epoch 6/12
60000/60000 [==============================] - 169s - loss: 0.0706 - acc: 0.9787 - val_loss: 0.0376 - val_acc: 0.9868
Epoch 7/12
60000/60000 [==============================] - 169s - loss: 0.0650 - acc: 0.9808 - val_loss: